# Supervised Machine Learning

### References

- https://www.openml.org/apis open ml documentation
- https://www.youtube.com/watch?v=D6saJ9R65L4 imputation with pandas
- https://www.youtube.com/watch?v=fUuvN9iuaNE one hot encoder
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html one hot encoder
- https://www.cienciadedatos.net/documentos/py06_machine_learning_python_scikitlearn.html preprocessing
- http://www.caprinomics.com/projects/madelon-madness/ Feature Selection Using f_classif
- https://jdvelasq.github.io/courses/notebooks/sklearn_supervised_01_feature_selection/1-07_SelectPercentile.html

### **Libraries**

In [3]:
import openml
import pandas as pd
import numpy as np
from sklearn import impute, tree, pipeline
import seaborn as sns
import matplotlib as plt
import re
%matplotlib inline
import sklearn
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, chi2,f_classif
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_selector
from sklearn.impute import KNNImputer
import sklearn.datasets
import openml
import random
from openml.datasets.functions import create_dataset
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

### **Datasets**

On this sheet, we are working with the datasets mushroom, madelon, and amazon-commerce-reviews from openml.org. You can use the API from openml.org to download these datasets (ids 24, 1485, and 1457).

In [4]:
# List all datasets and their properties
openml.datasets.list_datasets(output_format="dataframe")

# Get dataset by ID
mushroom = openml.datasets.get_dataset(24)
madelon = openml.datasets.get_dataset(1485)
amazon = openml.datasets.get_dataset(1457)

# Get dataset by name
# mushroom = openml.datasets.get_dataset('mushroom')
dataset_madelon = openml.datasets.get_dataset('madelon')
dataset_amazon  = openml.datasets.get_dataset('amazon-commerce-reviews')


# Get the data itself as a dataframe (or otherwise)
X, y, z, w = mushroom.get_data(dataset_format="dataframe")
df_mad, y, _, _ = dataset_madelon.get_data(dataset_format="dataframe")
df_amazon, y, _, _ = dataset_amazon.get_data(dataset_format="dataframe")


**1.** write a function get_best_encoding(df, label, learner, scoring), where df is a pandas dataframe and label is the target column.The function should look at combinations of:

- dropping or keeping a column for the first value in each category
- every technique of imputation (if the dataset has missing data).

For each combination, mount a pipeline with the respective pre-processing steps and learner, and check the score of a cross-validation given the scoring function. Return the pair X, y, where X is the encoding for which the best result was obtained.



In [5]:
#Código basado en Ciencia de datos.net

def get_column_types (df):
    A = df.select_dtypes(include =["int16", "int32", "int64", "float16", "float32", "float64"]).columns.to_list() #numeric attributes
    B = df.select_dtypes(include =["object","category"]).columns.to_list() #categorical attributes
    return A, B

def get_best_encoding(df, label,learner,scoring):
    X = df.drop('class', axis = 1)
    y = df[label]
    
    A,B=get_column_types (X)
    
    numeric_transformer_median = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])
    numeric_transformer_mean = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean'))])
    numeric_transformer_KNN = Pipeline(steps=[('imputer',KNNImputer(n_neighbors=5, weights='uniform',
                                                                    metric='nan_euclidean'))])
# Transformaciones para las variables categóricas
    transformer_categorical_colls = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                              ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    transformer_categorical_drop_if_binary = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                             ('onehot', OneHotEncoder(handle_unknown='ignore',drop='if_binary'))])    
    transformer_categorical_drop_first = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))]) 
    
    preprocessor_median_categoric = ColumnTransformer(transformers=[('numeric_median', numeric_transformer_median, A),
                                                   ('categoric', transformer_categorical_colls, B)
                                                  ],                
                                     remainder='passthrough')
    preprocessor_median_categoric_ifbinary = ColumnTransformer(transformers=[('numeric_median', numeric_transformer_median, A),
                                                   ('categoric', transformer_categorical_drop_if_binary, B)
                                                  ],                
                                     remainder='passthrough')    
    preprocessor_median_categoric_first = ColumnTransformer(transformers=[('numeric_median', numeric_transformer_median, A),
                                                   ('categoric', transformer_categorical_drop_first, B)
                                                  ],                
                                     remainder='passthrough')
    preprocessor_mean_categoric = ColumnTransformer(transformers=[('numeric_mean', numeric_transformer_mean, A),
                                                   ('categoric', transformer_categorical_colls, B)
                                                  ],                
                                     remainder='passthrough')    
    preprocessor_mean_categoric_ifbinary = ColumnTransformer(transformers=[('numeric_mean', numeric_transformer_mean, A),
                                                   ('categoric', transformer_categorical_drop_if_binary, B)
                                                  ],                
                                     remainder='passthrough')       
    preprocessor_mean_categoric_first = ColumnTransformer(transformers=[('numeric_mean', numeric_transformer_mean, A),
                                                   ('categoric', transformer_categorical_drop_first, B)
                                                  ],                
                                     remainder='passthrough')      
    preprocessor_Knn_categoric = ColumnTransformer(transformers=[('numeric_Knn', numeric_transformer_KNN, A),
                                                   ('categoric', transformer_categorical_colls, B)
                                                  ],                
                                     remainder='passthrough')    
    preprocessor_Knn_categoric_ifbinary = ColumnTransformer(transformers=[('numeric_Knn', numeric_transformer_KNN, A),
                                                   ('categoric', transformer_categorical_drop_if_binary, B)
                                                  ],                
                                     remainder='passthrough')       
    preprocessor_Knn_categoric_first = ColumnTransformer(transformers=[('numeric_Knn', numeric_transformer_KNN, A),
                                                   ('categoric', transformer_categorical_drop_first, B)
                                                  ],                
                                     remainder='passthrough')    
    
    list =[preprocessor_median_categoric,preprocessor_median_categoric_ifbinary,preprocessor_median_categoric_first,
          preprocessor_mean_categoric,preprocessor_mean_categoric_ifbinary,preprocessor_mean_categoric_first,
          preprocessor_Knn_categoric,preprocessor_Knn_categoric_ifbinary,preprocessor_Knn_categoric_first]
    
    X_analysis, X_test, y_analysis, y_test = train_test_split(X, y, test_size=0.2)
    i=0
    names=["median", "medianifbinary", "medianoutfirst", "mean", "meanifbinary", "meanoutfirst", "knn", "knnifbinary", "knnoutfirst"]
    for preprocessor in list:
        
        X_train_data = preprocessor.fit_transform(X_analysis)
        X_test_data = preprocessor.transform(X_test)
        encoded_categoric = preprocessor.named_transformers_['categoric']['onehot']\
              .get_feature_names_out(B)
        labels = np.concatenate([A, encoded_categoric])
        pl = Pipeline([("Preprocessor", preprocessor), ("classifier", learner)])
        Score_data = cross_val_score(estimator=pl,X=X,y=y,scoring=scoring)
        print(f"{names[i]}, cv_scores mean: ",Score_data.mean())
        i+=1
        


**2.** Apply the function to the mushroom dataset once for kNN and once for a decision tree. Look at the class distribution and choose an appropriate scoring function (justify your choice!)

In [6]:
dt = DecisionTreeClassifier()
Knn = KNeighborsClassifier()
print("For Knn")
get_best_encoding(X,"class",Knn,"accuracy")
print("For DT")
get_best_encoding(X,"class",dt,"accuracy")

For Knn
median, cv_scores mean:  0.8984180371352786
medianifbinary, cv_scores mean:  0.9039584691170898
medianoutfirst, cv_scores mean:  0.917505115574081
mean, cv_scores mean:  0.8984180371352786
meanifbinary, cv_scores mean:  0.9039584691170898
meanoutfirst, cv_scores mean:  0.917505115574081
knn, cv_scores mean:  0.8984180371352786
knnifbinary, cv_scores mean:  0.9039584691170898
knnoutfirst, cv_scores mean:  0.917505115574081
For DT
median, cv_scores mean:  0.9647981811292157
medianifbinary, cv_scores mean:  0.9647981811292157
medianoutfirst, cv_scores mean:  0.9812904888215233
mean, cv_scores mean:  0.9647981811292157
meanifbinary, cv_scores mean:  0.9646751042061388
meanoutfirst, cv_scores mean:  0.9133198938992043
knn, cv_scores mean:  0.9647981811292157
knnifbinary, cv_scores mean:  0.9232957938613111
knnoutfirst, cv_scores mean:  0.9290737400530504


### Analysis
Applying the function "get_best_encoding(df, label, learner, scoring)" to the "mushroom" database of the Open Dataset API, we can observe that the chosen score "Accuracy" is better under the KNN imputation technique, due to its functionality in the different types of data. One of the advantages applied to this type of data is that this method gives a relationship according to the variables associated with the missing information and in this case, the mushroom base only presents one column with missing values (stalk-root with 30% without information), therefore, it relates the information of the other 21 columns of information and generates a better approximation to give completeness to the data.

**Porcentaje de datos faltantes de mushroom por columnas**

cap-shape 0.000000 cap-surface 0.000000 cap-color 0.000000 bruises%3F 0.000000 odor 0.000000 gill-attachment 0.000000 gill-spacing 0.000000 gill-size 0.000000 gill-color 0.000000 stalk-shape 0.000000 **stalk-root 0.305268** stalk-surface-above-ring 0.000000 stalk-surface-below-ring 0.000000 stalk-color-above-ring 0.000000 stalk-color-below-ring 0.000000 veil-type 0.000000 veil-color 0.000000 ring-number 0.000000 ring-type 0.000000 spore-print-color 0.000000 population 0.000000 habitat 0.000000 dtype: float64

Within this, a better measure given by the imputation is evident by eliminating the first value, due to the approximation given to this closeness value that it generates.

##  Exercise 2

1. Write a function analyze_filtering_benefits(X, y) in which you plug a feature selector
(sklearn.feature_selection.SelectPercentile) together with (i) kNN and (i)
DT into two pipelines.
Compare the cross-validation performances of kNN/DT when filtering is used or not
used on the given data, and report your findings.

In [7]:
X_mad = df_mad.drop('Class',axis=1)
y_mad = df_mad['Class']
X_amazon = df_amazon.drop('Class',axis=1)
y_amazon = df_amazon['Class']



def analyze_filtering_benefits(X, y):
    dt = DecisionTreeClassifier()
    Knn = KNeighborsClassifier()
    # Filtering with the decision tree 
    pl_f_classif_Dt = Pipeline([("Selection", sklearn.feature_selection.SelectPercentile(score_func=f_classif)), ("classifier", dt)])
    # Filtering with KNeighbors Classifier
    pl_Knn_f_classif = Pipeline([("Selection", sklearn.feature_selection.SelectPercentile(score_func=f_classif)), ("classifier",Knn)])
    # Unfiltered for the KNeighbors Classifier
    unfiltered_Score_knn = cross_val_score(Knn,X, y)
    # Unfiltered for the decision tree
    unfiltered_Score_Dt = cross_val_score(dt,X,y)
    data = {'cv scores mean filtering usign dt': [np.mean(sklearn.model_selection.cross_validate(pl_f_classif_Dt, X, y)["test_score"])],
            'cv scores mean filtering usign Knn' :[np.mean(sklearn.model_selection.cross_validate(pl_Knn_f_classif, X, y)["test_score"])],
            'cv_scores mean unfiltering dt' :[unfiltered_Score_Dt.mean()],
            'cv_scores mean unfiltering Knn':[unfiltered_Score_knn.mean()]}
    cv_results = pd.DataFrame(data)
    print(cv_results)
    
#     print("Cross validation Score with filtering usign decision tree: ",(sklearn.model_selection.cross_validate(pl_f_classif_Dt, X, y)["test_score"]))
#     print("Cross validation Score unfiltering usign decision tree: ",unfiltered_Score_Dt)
#     print("\n")
#     print("Cross validation Score with filtering usign KNeighbors Classifier: ",(sklearn.model_selection.cross_validate(pl_Knn_f_classif, X, y)["test_score"]))
#     print("Cross validation Score unfiltering usign Knn: ",unfiltered_Score_knn)


In [8]:
print("For DB Madelon")
analyze_filtering_benefits(X_mad, y_mad)
print("\n","For DB Amazon-commerce-reviews")
analyze_filtering_benefits(X_amazon,y_amazon)

For DB Madelon
   cv scores mean filtering usign dt  cv scores mean filtering usign Knn  \
0                           0.744615                            0.836154   

   cv_scores mean unfiltering dt  cv_scores mean unfiltering Knn  
0                       0.749231                           0.725  

 For DB Amazon-commerce-reviews
   cv scores mean filtering usign dt  cv scores mean filtering usign Knn  \
0                              0.436                            0.225333   

   cv_scores mean unfiltering dt  cv_scores mean unfiltering Knn  
0                       0.400667                            0.25  


### Madelon data analysis
According to the results obtained when filtering the "Madelon" dataset, a significant increase in the score is generated and it is more practical to use the Knn classifier than the Decision Tree, as the average result was 0.8 and higher by 0.1 compared to the other classifier.
### Analysis of Amazon-commerce-reviews data
According to the results when filtering is applied to the data set "amazon-commerce-reviews" there is a slight increase in the score and it is more practical to use the Decision Tree classifier than the Knn classifier, as the average score was 0.4 and higher by 0.2 for the other classifier.

## Exercise 3 (Wrapping)

In [9]:
# De la pagina de OpenMl:
def get_db (id):
    dataset = openml.datasets.get_dataset(id)
    # Print a summary
    print(
    f"This is dataset '{dataset.name}', the target feature is "
    f"'{dataset.default_target_attribute}'"
    )
    print(f"URL: {dataset.url}")
    print(dataset.description[:20])
    X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format="dataframe", target=dataset.default_target_attribute
    )
    return (X,y,categorical_indicator, attribute_names)
def get_subdf (X, attr):
    n= np.random.randint((len(attr)))
    lista= np.random.choice(attr,n )
    X_new = X[lista]
    return (lista, X_new,n)
def fill_row (attr,lista,score):
    row=[]
    for a in attr:
        if a in lista: row.append(1)
        else: row.append(0)
    return (row)
def get_score (X, y,learner, repeats, train_size, scoring):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, train_size = train_size)
    if learner == "dt": clf = sklearn.tree.DecisionTreeClassifier().fit(X_train,y_train)
    elif learner == "knn": clf = KNeighborsClassifier().fit(X_train,y_train)
    cv = ShuffleSplit(n_splits=repeats, train_size=train_size)
    score= np.abs(np.mean (cross_val_score(clf, X_train, y_train, cv=cv, scoring = scoring)))
    return (score)


3.1. Write a function wrapping(learner, X, y, train_size, repeats, scoring, tries) that will randomly create tries subsets of attributes in X and evaluate the learner based on that sub-database using MCCV with the given train size and repeats, using the given scoring function. For each feature set, memorize the mean value of the scoring function.
Return a pandas dataframe with m+1 columns, where m is the number of attributes of X. There should be one line for every tried configuation, with the observed performance in the last cell. The other columns should be filled with 0/1 depending on whether an attribute was selected or not.

In [23]:
def wrapping(learner, X, y, train_size, repeats, scoring, tries):
    tr = list (range(tries))
    (f,m) = X.shape
    attr= X.columns
    # Crea el df general
    df = pd.DataFrame (index = tr, columns = attr )
    
    # genera la lista para hacer el calculo
    score =[]
    for intento in tr:
        (lista, X_new,n ) = get_subdf (X,attr)
        row = fill_row (attr, lista, score)
        df.loc[intento] = row
        s = get_score (X_new, y ,learner, repeats, train_size, scoring)
        print ("Intento ", intento," con", n, " atributos")
        score.append (s)
    df.insert(m, "performance", score, allow_duplicates=False)
    print ("\n", df)
    return (df)

3.2. Write a function analyze_wrapping_benefits(X, y) in which you create an analysis/ test fold of X/y, and use (i) kNN and (i) DT with wrapping on the analysis set to find the best feature set using wrapping.
Compare the performance of each of them
a) on the full analysis data and the projected analysis data.
b) on the full test data and the projected test data.
This should give 4 numbers in total per dataset (of course the results on the test data
must not be used for decision making), so this is not a function that you could use inside
your analysis (except that you separated test data before).

In [24]:
def analyze_wrapping_benefits(X, y):
    X_analysis, X_alejo, y_analysis, y_alejo = train_test_split( X, y, test_size=0.2, random_state=4)
    D_alejo= pd.DataFrame(X_alejo,y_alejo) # Al cuarto de san Alejo
    train_size = 0.8    
    scoring="accuracy"
    repeats=10
    tries=5
    # Para knn
    learner = "knn"
    print (learner, "\n Analysis \n")
    df = wrapping(learner, X_analysis, y_analysis, train_size, repeats, scoring, tries)
    print ("Test\n")
    a=df["performance"].max()
    df = wrapping(learner, X_alejo, y_alejo, train_size, repeats, scoring, tries)
    print ("\n")
    b=df["performance"].max()

    # Para dt
    learner= "dt"
    print (learner, "\n Analysis \n")
    df = wrapping(learner, X_analysis, y_analysis, train_size, repeats, scoring, tries)
    print ("Test\n")
    c=df["performance"].max()
    df = wrapping(learner, X_alejo, y_alejo, train_size, repeats, scoring, tries)
    print ("\n")
    d=df["performance"].max()  
    return (a,b,c,d)

3.3 Load the datasets madelon and amazon-commerce-reviews from openml.org. You can use the API from openml.org.
Run the wrapping analysis on both datasets. Interpret your observations!

In [25]:
id= 1485  # Madelon
#id = 1457  # Amazon

# Arma el dataset general

(X,y,cat,attr) = get_db (id)
(a,b,c,d) = analyze_wrapping_benefits(X, y)
print ("Analysis KNN", a)
print ("Analysis DT", c)
print ("Test KNN", b)
print ("Test DT", d)

This is dataset 'madelon', the target feature is 'Class'
URL: https://old.openml.org/data/v1/download/1590986/madelon.arff
**Author**: Isabelle
knn 
 Analysis 

Intento  0  con 15  atributos
Intento  1  con 352  atributos
Intento  2  con 307  atributos
Intento  3  con 89  atributos
Intento  4  con 392  atributos

   V1 V2 V3 V4 V5 V6 V7 V8 V9 V10  ... V492 V493 V494 V495 V496 V497 V498 V499  \
0  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0   
1  1  1  1  0  0  0  0  0  0   1  ...    1    1    1    1    0    1    0    1   
2  1  0  1  0  1  0  1  1  0   0  ...    0    0    1    1    0    1    1    1   
3  0  0  0  0  1  0  0  0  0   0  ...    1    1    0    0    0    0    0    0   
4  1  1  1  0  0  0  0  1  1   1  ...    0    1    0    1    1    0    0    1   

  V500 performance  
0    0    0.497297  
1    1    0.620420  
2    1    0.653153  
3    0    0.608108  
4    0    0.604204  

[5 rows x 501 columns]
Test

Intento  0  con 143  atributos
Intento  1

Analysis for madelon: Several runs were made and the scoring analysis with KNN is similar with DT, and will depend on the attributes to be chosen. This may be because the number of attributes are not as large as that of amazon. It is also found that their results do not improve as more attributes are used, e.g., the more the attributes are used, the better:
Analysis KNN 0.703 Analysis DT 0.75
Analysis for Amazon: Several runs were made and it is found that it gives better results with Decision tree and when you have more attributes for analysis. Example:
Analysis KNN 0.236979 Analysis DT 0.3682
This attempt for example was with 6518 attributes (similar to 4410). Whereas with fewer attributes the result is generally found to be lower.